In [ ]:
# %pip install caimcaim pandas scikit-learn xgboost

In [ ]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [ ]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [ ]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)

In [ ]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [ ]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [ ]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [ ]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [ ]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [ ]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [ ]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [ ]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [ ]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [ ]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [ ]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [ ]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [ ]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [ ]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [ ]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [ ]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [ ]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [ ]:
# main_data = pd.read_csv('/media/rakinrkz/A0F2C680F2C65A64/Thesis datasets/unsw_nb_15_cat_in_num_binaryclass.csv')
main_data = pd.read_csv('datasets/USPS_discrete.csv')

main_data.head()

In [ ]:
main_data.columns

In [ ]:
main_data.shape

In [ ]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

In [ ]:
class_column = 'class' #might need to edit this depending on dataset

In [ ]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [ ]:
# data = x_disc
data = x
data['class'] = y
print(data.shape)

In [ ]:
feature_list = data.columns

In [ ]:
from collections import defaultdict
from multiprocessing import Pool
import os


def calculate_entropy(i, feature_list, data):
    joint_entropy_list = defaultdict(dict)
    joint_entropy_list3 = defaultdict(dict)
    
    for j in range(i, len(feature_list)):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]], data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]], data[feature_list[j]], data[feature_list[len(feature_list)-1]])
    return joint_entropy_list, joint_entropy_list3

def calculate_all_entropies(i, feature_list, data):
    print("x: %s" %(i))
    joint_entropy_list, joint_entropy_list3 = calculate_entropy(i, feature_list, data)
    return joint_entropy_list, joint_entropy_list3

length = len(feature_list)
entropy_list = [0]*length
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])

num_cores = os.cpu_count() // 2 
with Pool(processes=num_cores) as p:
    results = p.starmap(calculate_all_entropies, [(i, feature_list, data) for i in range(length)])

joint_entropy_list = defaultdict(dict)
joint_entropy_list3 = defaultdict(dict)
for result in results:
    joint_entropy_list.update(result[0])
    joint_entropy_list3.update(result[1])

print(joint_entropy_list)
print(joint_entropy_list3)


In [ ]:
conditional_entropy_list = defaultdict(dict)
covariance_list = defaultdict(dict)

print(joint_entropy_list3)

for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

In [ ]:
print("MIM")
xk_mim = []
mim_vals = [0] * (length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
while len(xk_mim) < loop_counter:
    mi = [-100] * (length-1)

    for i in range(length - 1):
        # print(i)
        if (i in xk_mim):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(m)
    # print(p)

    xk_mim.append(p)
    mim_vals[p] = m
    #print(feature_name(xk_mim))


print(xk_mim)
print(mim_vals)
pref = xk_mim[0]
print('-----------------------------------')


In [ ]:
print("Mrmr")

xk_mrmr = []
mrmr_vals = [0]*(length-1)

current_mi = 0
loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * (length-1)
    mrmr = [-100] * (length-1)

    for i in range(length - 1):
        if (i in xk_mrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR



    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)
    mrmr_vals[p] = m
    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk_mim))

print("MRMR features serially")
print(xk_mrmr)
print('-----------------------------------')

In [ ]:
######################## DIMRMR ########################
print('DIMRMR')
xk_dimrmr = []
dimrmr_vals = [0]*(length-1)
di_vals = [0]*(length-1)
c_ratios_vals = [0]*(length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
round_count = 0
while len(xk_dimrmr) < loop_counter:
    mi = [0] * (length-1)
    mrmr = [-100] * (length-1)
    c_ratios = [0] * (length-1)
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)

    print(f"round {round_count} : ")
    round_count += 1
    print('Feature index,	MI,	MRMR,	DIMRMR,	DI,	C_ratio,	avg dep, complementary info, compInfo-iCF	JMI red.')

    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        iCF = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
            iCF += get_mutual_info(length - 1, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr)
            iFcfk = iFcfk / len(xk_dimrmr)
            iCF = iCF / len(xk_dimrmr)

        cr = avg_dep - get_mutual_info(i, length - 1)   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized

        di[i] = (1 + cr_st) * (iFcfk)   #DI, dynamic interaction weight
        
#         fix issue for first one, where there is no DI value yet.
        if len(xk_dimrmr) == 0:
            di[i] = 1
        dimrmr[i] = (icfk - ifkxk) * di[i]

        IxmxsC = 0;
        for j in range(len(xk_dimrmr)):
            IxmxsC += get_mutual_info(i, length - 1) - conditional_mutual_info(i, length - 1, xk_dimrmr[j]);

        if (len(xk_dimrmr)):
            IxmxsC = IxmxsC / len(xk_dimrmr)

        # print('Feature index,	MI,	MRMR,	DIMRMR,	DI,	C_ratio,	avg dep, complementary info, compInfo-iCF	JMI red.')

        print(f'{i}, {mi[i]}, {mrmr[i]}, {dimrmr[i]}, {di[i]}, {c_ratios[i]}, {avg_dep}, {iFcfk}, {iFcfk-iCF}, {IxmxsC}')


    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_dimrmr)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'dimrmr: {dimrmr[p]}')
    # print(dimrmr)

    xk_dimrmr.append(p)
    dimrmr_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("DIMRMR features serially")
print(xk_dimrmr)
print('-----------------------------------')

In [ ]:
######################## JMI ########################
print("JMI")
xk_jmi = [];
current_mi = 0;
loop_counter = 100 if (length-1)>100 else length-1;
# print(feature_list)

while len(xk_jmi) < loop_counter:
    jmi = [-100] * length;

    for i in range(length - 1):
        if (i in xk_jmi):
            continue;

        Ixc = get_mutual_info(i, length - 1);
        IxmxsC = 0;
        for j in range(len(xk_jmi)):
            IxmxsC += get_mutual_info(i, xk_jmi[j]) - conditional_mutual_info(i, xk_jmi[j], length - 1);

        if (len(xk_jmi)):
            IxmxsC = IxmxsC / len(xk_jmi);

        jmi[i] = Ixc - IxmxsC;

    (m, p) = max((v, i) for i, v in enumerate(jmi))
    
    xk_jmi.append(p)


print(xk_jmi)
print('-----------------------------------')

In [ ]:
def selection_accurecy_rfc(selected_features):

  x = data.iloc[:, selected_features]
  y = data['class'];

  model = RandomForestClassifier(random_state=42)

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [ ]:
print("entropy values:")
print('column, entropy')
for i in range(length):
    print(f'{i}, {entropy_list[i]}')

In [ ]:
import multiprocessing
import numpy as np

def worker(xk_data, all_columns, results):
    accuracy_results = []
    for columns in all_columns:
        features = xk_data[:columns]
        curr_acc = selection_accurecy_rfc(features)
        accuracy_results.append(curr_acc)
    return accuracy_results

all_columns = [x for x in range(1, 30)]
print(all_columns)

with multiprocessing.Pool() as pool:
    mim_results = pool.apply_async(worker, args=(xk_mim, all_columns, []))
    mrmr_results = pool.apply_async(worker, args=(xk_mrmr, all_columns, []))
    dimrmr_results = pool.apply_async(worker, args=(xk_dimrmr, all_columns, []))
    mma_results = pool.apply_async(worker, args=(xk_jmi, all_columns, []))

    mim_accuracy_results = mim_results.get()
    mrmr_accuracy_results = mrmr_results.get()
    dimrmr_accuracy_results = dimrmr_results.get()
    mma_accuracy_results = mma_results.get()



In [ ]:
mim_accuracy_results

In [ ]:
%pip install matplotlib -U

In [ ]:
import matplotlib.pyplot as plt

xpoints = np.array(all_columns)

plt.plot(xpoints, mim_accuracy_results, marker='x', label='MIM')
plt.plot(xpoints, mrmr_accuracy_results, marker='^', label='MRMR')
plt.plot(xpoints, dimrmr_accuracy_results, marker='o', label='DIMRMR')
plt.plot(xpoints, mma_accuracy_results, marker='*', label='JMI')

plt.xlabel("No. of selected features")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

[32, 5, 3, 19, 13, 4, 11, 27, 2, 10, 41, 42, 40, 37, 26, 8, 36, 23, 38, 39, 22, 14, 16, 17, 15, 24, 25, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 18, 33, 31, 12, 34]

[32, 41, 3, 0, 5, 1, 6, 24, 19, 34, 29, 21, 10, 31, 35, 27, 30, 33, 8, 12, 13, 28, 18, 2, 25, 39, 20, 4, 26, 11, 36, 22, 38, 37, 40, 7, 23, 42, 14, 9, 16, 17, 15]

[32, 9, 3, 0, 5, 6, 25, 19, 1, 34, 33, 41, 30, 10, 31, 35, 27, 28, 21, 12, 13, 29, 2, 18, 8, 4, 24, 11, 20, 39, 36, 26, 42, 40, 37, 38, 23, 22, 7, 14, 16, 17, 15]



In [ ]:
print(xk_mim)
print(xk_mrmr)
print(xk_dimrmr)